In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_NCD.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CCY = MktSheet.cell(5, 1).value
Type = MktSheet.cell(5, 2).value
MTMDate = MktSheet.cell(1, 3).value
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted

In [4]:
NCDSP = MktSheet.cell(13, 8).value
NCD3M = MktSheet.cell(14, 8).value
NCD6M = MktSheet.cell(15, 8).value
NCD12M = MktSheet.cell(16, 8).value
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date12M = Cal.advance(evDate, 12, ql.Months, BDC, False)
ListDate = list([DateSP, Date3M, Date6M, Date12M])
ListRate = list([NCDSP, NCD3M, NCD6M, NCD12M])
VecDate = ql.DateVector(ListDate)
VecRate = ql.DoubleVector(ListRate)
TSZero = ql.ZeroCurve(VecDate, VecRate, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)

In [5]:
NCDSheet = PstBook['NCD']
Rows = NCDSheet.rows
TRows = tuple(Rows) # 6
start_row = 2 # First Record
end_row = len(TRows) # 6, Last Record
start_col = 0 # 'A' column index
end_col = 4 # 'E' column index
total_record = end_row - start_row + 1 # 6-2+1 = 5 Records
ListNCD = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in NCDSheet[i][start_col:end_col + 1]]
    ListNCD.append(row)
    print(row) # list of cell values of this row

['NCD001', datetime.datetime(2021, 6, 18, 0, 0), datetime.datetime(2021, 9, 17, 0, 0), 91, 200000000]
['NCD002', datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2021, 7, 21, 0, 0), 28, 700000000]
['NCD003', datetime.datetime(2021, 6, 25, 0, 0), datetime.datetime(2021, 7, 23, 0, 0), 28, 200000000]
['NCD004', datetime.datetime(2021, 6, 29, 0, 0), datetime.datetime(2021, 7, 27, 0, 0), 28, 1000000000]
['NCD005', datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2021, 7, 28, 0, 0), 28, 200000000]


In [6]:
NCDBook = list()
for i in range(len(ListNCD)):
    row = ListNCD[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(row[3])
    record.append(row[4])
    NCDBook.append(record)

In [7]:
NCDBook

[['NCD001', Date(18,6,2021), Date(17,9,2021), 91, 200000000],
 ['NCD002', Date(23,6,2021), Date(21,7,2021), 28, 700000000],
 ['NCD003', Date(25,6,2021), Date(23,7,2021), 28, 200000000],
 ['NCD004', Date(29,6,2021), Date(27,7,2021), 28, 1000000000],
 ['NCD005', Date(30,6,2021), Date(28,7,2021), 28, 200000000]]

In [8]:
h_TS = ql.YieldTermStructureHandle(TSZero)
NCDEngine = ql.DiscountingBondEngine(h_TS)

In [9]:
ListZero = ListRate
VecDate = ql.DateVector(ListDate)
ListListZero = list()
ListListZero.append(ListZero)

In [10]:
ListZero

[0.001194941316666515,
 0.001140835958777265,
 0.001156664550586739,
 0.00121725883976101]

In [11]:
NoCurve = len(ListZero)
print(NoCurve)
ListRW = list([0.017, 0.017, 0.016, 0.013, 0.012, 0.011, 0.011, 0.011, 0.011, 0.011])

4


In [12]:
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] + 0.0001)
        else:
            NewList.append(ListZero[j])
    ListListZero.append(NewList)

In [13]:
ListListZero

[[0.001194941316666515,
  0.001140835958777265,
  0.001156664550586739,
  0.00121725883976101],
 [0.001194941316666515,
  0.001240835958777265,
  0.001156664550586739,
  0.00121725883976101],
 [0.001194941316666515,
  0.001140835958777265,
  0.001256664550586739,
  0.00121725883976101],
 [0.001194941316666515,
  0.001140835958777265,
  0.001156664550586739,
  0.00131725883976101]]

In [14]:
ListVecZero = list()
ListTSZero = list()
for i in range(NoCurve):
    AVecZero = ql.DoubleVector(ListListZero[i])
    ListVecZero.append(AVecZero)
    ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
    ListTSZero.append(ATSZero)

In [15]:
ListTSZero

[<QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001C40D940A50> >,
 <QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001C40D940BD0> >,
 <QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001C40D940CF0> >,
 <QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001C40D940E40> >]

In [16]:
CurvUpListListZero = list()
CurvUpListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] + 0.017)
        else:
            NewList.append(ListZero[j])
    CurvUpListListZero.append(NewList)

In [17]:
CurvDnListListZero = list()
CurvDnListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] - 0.017)
        else:
            NewList.append(ListZero[j])
    CurvDnListListZero.append(NewList)

In [18]:
CurvUpListListZero[0]

[0.001194941316666515,
 0.001140835958777265,
 0.001156664550586739,
 0.00121725883976101]

In [19]:
CurvUpListListZero[1]

[0.001194941316666515,
 0.018140835958777267,
 0.001156664550586739,
 0.00121725883976101]

In [20]:
SenSheet = PstBook['GIRR']
for i in range(0, len(NCDBook)):
    ListValue = list()
    record = NCDBook[i]
    startDate = record[1]
    maturityDate = record[2]
    faceAmt = record[4]
    NCD = ql.ZeroCouponBond(0, Cal, faceAmt, maturityDate, BDC, faceAmt)
    
    h_TS = ql.YieldTermStructureHandle(ListTSZero[0])
    NCDEngine = ql.DiscountingBondEngine(h_TS)
    NCD.setPricingEngine(NCDEngine)
    Value = NCD.dirtyPrice()
    
    ListValue.append(Value)
    for j in range(1, NoCurve):
        h_TS = ql.YieldTermStructureHandle(ListTSZero[j])
        NCDEngine = ql.DiscountingBondEngine(h_TS)
        NCD.setPricingEngine(NCDEngine)
        Value = NCD.dirtyPrice()
        ListValue.append(Value)
        SenSheet.cell(i+5, 1).value = record[0]
        SenSheet.cell(i+5, 2).value = ListValue[0]      
    
    ListSensitivity = list()
    for j in range(1, len(ListValue)):
        ListSensitivity.append((ListValue[j] - ListValue[0])*10000)

    ListWS = list()
    for j in range(0, len(ListSensitivity)):
        ListWS.append(ListRW[j] * ListSensitivity[j])
    
    for j in range(0, len(ListSensitivity)):
        SenSheet.cell(i+5, j+5).value = ListWS[j]
    
    ListVecZero = list()
    CurvUpListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvUpListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvUpListTSZero.append(ATSZero)
                
    ListVecZero = list()
    CurvDnListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvDnListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvDnListTSZero.append(ATSZero)
        
    CurvUpListHTSZero = list()
    for j in range(NoCurve):
        CurvUpListHTSZero.append(ql.YieldTermStructureHandle(CurvUpListTSZero[j]))
        
    CurvDnListHTSZero = list()
    for j in range(NoCurve):
        CurvDnListHTSZero.append(ql.YieldTermStructureHandle(CurvDnListTSZero[j]))
        
    CurvUpValue = list()
    for j in range(NoCurve):
        NCDEngine = ql.DiscountingBondEngine(CurvUpListHTSZero[j])
        NCD.setPricingEngine(NCDEngine)
        Value = NCD.dirtyPrice()
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoCurve):
        NCDEngine = ql.DiscountingBondEngine(CurvDnListHTSZero[j])
        NCD.setPricingEngine(NCDEngine)
        Value = NCD.dirtyPrice()
        CurvDnValue.append(Value)        
        
    CurvatureUp = list()
    ShiftUpValue = list()
    for j in range(1, NoCurve):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - 0.017 * ListSensitivity[j-1]))
                
    for j in range(0, len(CurvatureUp)):    
        SenSheet.cell(i+5, j+17).value = CurvatureUp[j]
            
    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoCurve):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])    
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + 0.017 * ListSensitivity[j-1]))

    for j in range(0, len(CurvatureDn)):    
        SenSheet.cell(i+5, j+29).value = CurvatureDn[j]
        
PstBook.save('Pst_NCD.xlsx')
PstBook.close()

In [21]:
CurvatureUp

[-15.657028824090958, -0.0, -0.0]

In [22]:
CurvatureDn

[-15.846508711576462, -0.0, -0.0]